# Verify effect of Hidden Similarity Regularization on Models

In [1]:
import pandas as pd

hidden_reg_df = pd.read_csv("../results/hidden_reg.csv", delimiter=";")
hidden_reg_df

,algorithm,reg_0,reg_0.1,reg_0.5
0,activity_selector,0.9098,0.9130,0.87180
1,articulation_points,0.9976,0.5076,0.97595
2,bellman_ford,0.9663,0.9829,0.98100
3,bfs,0.9990,0.9961,0.99320
4,binary_search,0.2813,0.3438,0.40630
5,bridges,0.8704,0.8751,0.72700
6,bubble_sort,0.1254,0.2236,0.28960
7,dag_shortest_paths,0.9917,0.9888,0.98680
8,dfs,0.1909,0.5498,0.19140
9,dijkstra,0.9888,0.9819,0.98630


# Load Datasets

In [2]:
import torch
import yaml
from algo_reasoning.src.sampler import CLRSDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/Users/ronaldalbert/miniconda3/envs/algo_reasoning/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def load_algorithm_args(args_file):
    with open(args_file, 'r') as f:
        args = yaml.safe_load(f)

    return args

# Comparing effect of Hidden Regularization to Algorithm Length

In [4]:
algorithm_args = load_algorithm_args("../algorithm_args/default.yaml")
hidden_reg_df["max_length"] = pd.Series([0] * len(hidden_reg_df))

for alg in hidden_reg_df.algorithm:
    print("Generating sample for: ", alg)
    algorithms = [alg]
    nb_nodes = 64
    ds = CLRSDataset(algorithms, nb_nodes, 1, 1000, seed=7, algorithms_args=algorithm_args)
    obj = next(iter(ds)).to(device=device)

    hidden_reg_df.loc[hidden_reg_df.algorithm == alg, ["max_length"]] =  obj.max_length.item()

Generating sample for:  activity_selector
Generating sample for:  articulation_points
Generating sample for:  bellman_ford
Generating sample for:  bfs
Generating sample for:  binary_search
Generating sample for:  bridges
Generating sample for:  bubble_sort
Generating sample for:  dag_shortest_paths
Generating sample for:  dfs
Generating sample for:  dijkstra
Generating sample for:  find_maximum_subarray_kadane
Generating sample for:  floyd_warshall
Generating sample for:  graham_scan
Generating sample for:  heapsort
Generating sample for:  insertion_sort
Generating sample for:  kmp_matcher
Generating sample for:  lcs_length
Generating sample for:  matrix_chain_order
Generating sample for:  minimum
Generating sample for:  mst_kruskal
Generating sample for:  mst_prim
Generating sample for:  naive_string_matcher
Generating sample for:  optimal_bst
Generating sample for:  quickselect
Generating sample for:  quicksort
Generating sample for:  segments_intersect
Generating sample for:  strong

In [5]:
reg_1_effect = hidden_reg_df["reg_0.1"] - hidden_reg_df["reg_0"]
reg_5_effect = hidden_reg_df["reg_0.5"] - hidden_reg_df["reg_0"]

hidden_reg_df["reg_0.1_effect"] = reg_1_effect
hidden_reg_df["reg_0.5_effect"] = reg_5_effect

In [6]:
_corr = hidden_reg_df[hidden_reg_df.columns.difference(['algorithm'])].corr()

_corr

,max_length,reg_0,reg_0.1,reg_0.1_effect,reg_0.5,reg_0.5_effect
max_length,1.000000,-0.001009,-0.048073,-0.126807,0.040936,0.127936
reg_0,-0.001009,1.000000,0.938425,-0.361715,0.953275,-0.392784
reg_0.1,-0.048073,0.938425,1.000000,-0.017353,0.927981,-0.266913
reg_0.1_effect,-0.126807,-0.361715,-0.017353,1.000000,-0.254671,0.416475
reg_0.5,0.040936,0.953275,0.927981,-0.254671,1.000000,-0.096606
reg_0.5_effect,0.127936,-0.392784,-0.266913,0.416475,-0.096606,1.000000


# Effect by Algorithm Type

In [7]:
type_dict = {
    "divide_and_conquer": ["find_maximum_subarray_kadane"],
    "dynamic_programming": ["matrix_chain_order", "lcs_length", "optimal_bst"],
    "geometry": ["segments_intersect", "graham_scan", "jarvis_march"],
    "graphs": ["dfs", "bfs", "topological_sort", "articulation_points", "bridges", "strongly_connected_components", "mst_kruskal", "mst_prim", "bellman_ford", "dijkstra", "dag_shortest_paths", "floyd_warshall"],
    "greedy": ["activity_selector", "task_scheduling"], 
    "searching": ["minimum", "binary_search", "quickselect"],
    "sorting": ["insertion_sort", "bubble_sort", "heapsort", "quicksort"],
    "strings": ["naive_string_matcher", "kmp_matcher"]
}

def get_algo_type(algo):
    for _type in type_dict.keys():
        print
        if algo in type_dict[_type]:
            return _type
        
hidden_reg_df["_type"] = hidden_reg_df.algorithm.apply(get_algo_type)
agg_df = hidden_reg_df[hidden_reg_df.columns.difference(['algorithm'])].groupby(['_type']).mean()
agg_df

,max_length,reg_0,reg_0.1,reg_0.1_effect,reg_0.5,reg_0.5_effect
_type,,,,,,
divide_and_conquer,64.0,0.140600,0.171900,0.031300,0.328100,0.187500
dynamic_programming,51.0,0.124400,0.103267,-0.021133,0.097767,-0.026633
geometry,62.5,0.954800,0.973100,0.018300,0.906200,-0.048600
graphs,360.0,0.782875,0.783400,0.000525,0.790129,0.007254
greedy,65.0,0.913600,0.890150,-0.023450,0.867900,-0.045700
searching,59.0,0.447933,0.416700,-0.031233,0.510467,0.062533
sorting,720.5,0.203475,0.162225,-0.041250,0.180800,-0.022675
strings,8.0,0.062500,0.203150,0.140650,0.125050,0.062550


In [8]:
agg_df.corr()

,max_length,reg_0,reg_0.1,reg_0.1_effect,reg_0.5,reg_0.5_effect
max_length,1.000000,-0.063028,-0.133453,-0.419833,-0.129989,-0.257190
reg_0,-0.063028,1.000000,0.988135,-0.315543,0.980033,-0.531258
reg_0.1,-0.133453,0.988135,1.000000,-0.166056,0.976803,-0.489170
reg_0.1_effect,-0.419833,-0.315543,-0.166056,1.000000,-0.257355,0.388718
reg_0.5,-0.129989,0.980033,0.976803,-0.257355,1.000000,-0.352194
reg_0.5_effect,-0.257190,-0.531258,-0.489170,0.388718,-0.352194,1.000000
